In [1]:
# clone the repo and change the working directory
! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6 ftfy==6.1.1

Cloning into 'video_features'...
remote: Enumerating objects: 1299, done.
remote: Counting objects: 100% (406/406), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 1299 (delta 251), reused 311 (delta 203), pack-reused 893
Receiving objects: 100% (1299/1299), 288.63 MiB | 21.21 MiB/s, done.
Resolving deltas: 100% (668/668), done.
Updating files: 100% (177/177), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.2 MB/s eta 0:00:00


In [2]:
%cd video_features

/content/video_features


In [82]:
from models.clip.extract_clip import ExtractCLIP
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch
import numpy as np
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#torch.cuda.get_device_name(0)

In [88]:
import os

#fpath = '/content/drive/MyDrive/video_feat'
fpath = '/content/drive/MyDrive/MultiModalDeepLearning/video_audio/videos'
textpath = "/content/drive/MyDrive/video_feat/video_fpath.txt"
filename_list = os.listdir(fpath)

video_fpath = []
for f in filename_list:
  if f.endswith((".mp4")):
    video_fpath.append(os.path.join(fpath, f))

open(textpath, 'w').close()
file_obj = open(textpath, "r+")

for vpath in video_fpath:
  #print(vpath)
  if vpath != textpath:
    file_obj.write(f'{vpath}\n')

file_obj.close()

In [90]:
print(len(video_fpath))

90


In [91]:
# Select the feature type
# ('CLIP-ViT-B-32', 'CLIP-ViT-B-16', 'CLIP-RN50x16', 'CLIP-RN50x4','CLIP-RN101', 'CLIP-RN50', 'CLIP-custom')
feature_type = 'clip'
model_name = 'ViT-B/32'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.feature_type = feature_type
args.video_paths = video_fpath
args.batch_size = 32
args.extraction_fps = 1
args.on_extraction = 'save_numpy'
args.output_path = '/content/drive/MyDrive/video_feat/features/'
# args.show_pred = True
# args.pred_texts = ['a dog smiles', 'a woman is lifting']  # if None, does zero-shot on Kinetics 400 classes

# Load the model
extractor = ExtractCLIP(args)

video_extrf = []
# Extract features
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    video_extrf.append(feature_dict['clip'])
    [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

Extracting for /content/drive/MyDrive/MultiModalDeepLearning/video_audio/videos/M-neskin-Beggin.mp4
clip
(213, 512)
[[ 5.34667969e-01  2.04833984e-01  4.39941406e-01 ...  2.22534180e-01
  -1.36230469e-01  1.10229492e-01]
 [ 3.26904297e-01  2.34985352e-01  1.97601318e-02 ...  3.29101562e-01
   1.83105469e-02  6.98242188e-02]
 [ 3.43750000e-01  6.26953125e-01  2.34863281e-01 ...  2.58056641e-01
  -2.73681641e-01 -7.23266602e-03]
 ...
 [ 2.16918945e-01  1.06567383e-01  2.63427734e-01 ...  3.66210938e-01
  -2.32238770e-02  1.82983398e-01]
 [-1.98974609e-01  1.36230469e-01 -3.78723145e-02 ...  8.47167969e-01
  -1.81884766e-02  3.07617188e-01]
 [-1.30004883e-01 -4.27246094e-04  3.54003906e-02 ...  6.82128906e-01
  -7.40356445e-02  2.17285156e-01]]
fps
()
1.0
timestamps_ms
(213,)
[     0.   1000.   2000.   3000.   4000.   5000.   6000.   7000.   8000.
   9000.  10000.  11000.  12000.  13000.  14000.  15000.  16000.  17000.
  18000.  19000.  20000.  21000.  22000.  23000.  24000.  25000.  2600

In [47]:
len(feature_dict['clip'][0])

512

In [102]:
cname = []
for f in video_fpath:
    pname = os.path.splitext(f)[0]
    fname = os.path.basename(pname)
    cname.append(fname)

print(cname)

['M-neskin-Beggin', 'Billie-Eilish-bad-guy', 'Taylor-Swift-The-Man', 'Olivia-Rodrigo-deja-vu', 'Olivia-Rodrigo-traitor', 'Zedd-Alessia-Cara-Stay', 'Conan-Gray-Heather', 'Lil-Nas-X-MONTERO-Call-Me-By-Your-Name-', 'Charlie-Puth-Left-And-Right', 'DJ-Khaled-I-m-The-One', 'Ed-Sheeran-Shape-of-You', 'The-Chainsmokers-Paris', '5-Seconds-of-Summer-Youngblood', 'GAYLE-abcdefu', 'Ti-sto-The-Business', 'Ti-sto-Ava-Max-The-Motto', 'Imagine-Dragons-x-J-I-D-Enemy', 'Dove-Cameron-Boyfriend', 'Elley-Duh-MIDDLE-OF-THE-NIGHT', 'Tate-McRae-you-broke-me-first', '24kGoldn-Mood', 'Kygo-Ellie-Goulding-First-Time', 'Lewis-Capaldi-Before-You-Go', 'Kali-Uchis-telepat-a', 'Camila-Cabello-Bam-Bam', 'Kygo-Selena-Gomez-It-Ain-t-Me', 'Doja-Cat-Woman', 'Imagine-Dragons-Believer', 'Jaymes-Young-Infinity', 'Ed-Sheeran-Galway-Girl', 'Calum-Scott-Dancing-On-My-Own', 'Calvin-Harris-Feels', 'Bruno-Mars-That-s-What-I-Like', 'Coldplay-X-BTS-My-Universe', 'Regard-Ride-It', 'Lil-Nas-X-Jack-Harlow-INDUSTRY-BABY', 'Joji-Glimpse-

In [106]:
# video_extrf dim: fps_length_ofvideo x 512 (clip features)

arr1 = np.array(video_extrf)

vfeatures = []
for i in range(len(arr1)):
  temp = np.mean(arr1[i], axis = 0)
  vfeatures.append(temp)

#print(vfeatures[0])
column_names = []
for f in filename_list:
  if f.endswith((".mp4")):
    column_names.append(f)

#df = pd.DataFrame(data=np.transpose(vfeatures), columns=column_names)

df2 = pd.DataFrame(data=np.transpose(vfeatures), columns=cname)
df2.to_csv('/content/drive/MyDrive/video_feat/features/video_features.csv', index=False)

<ipython-input-106-35eadbd6bad2>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr1 = np.array(video_extrf)


In [112]:
#df2.tail()
fpathaudio = '/content/drive/MyDrive/MultiModalDeepLearning/feature_matrix.csv'
fpathvideo = '/content/drive/MyDrive/video_feat/features/video_features.csv'

df_audio = pd.read_csv(fpathaudio)
df_video = pd.read_csv(fpathvideo)

df_audio = df_audio.sort_index(axis=1)
df_video = df_video.sort_index(axis=1)

In [118]:
df_audio.head()
df_video.head()

frames = [df_audio, df_video]
result = pd.concat(frames)

result.to_csv('/content/drive/MyDrive/video_feat/features/audioVideo_features.csv', index=False)
result.tail()

,24kGoldn-Mood,5-Seconds-of-Summer-Youngblood,Alec-Benjamin-Let-Me-Down-Slowly,Anne-Marie-2002,Axwell-and-Ingrosso-More-Than-You-Know,Billie-Eilish-Khalid-lovely,Billie-Eilish-bad-guy,Billie-Eilish-everything-i-wanted,Bruno-Mars-That-s-What-I-Like,Calum-Scott-Dancing-On-My-Own,...,Taylor-Swift-cardigan,The-Chainsmokers-Paris,The-Kid-LAROI-Justin-Bieber-STAY,The-Weeknd-Blinding-Lights,The-Weeknd-Out-of-Time,The-Weeknd-Save-Your-Tears,Ti-sto-10_35,Ti-sto-Ava-Max-The-Motto,Ti-sto-The-Business,Zedd-Alessia-Cara-Stay
507,0.220223,0.177679,0.027564,0.130196,0.078873,0.038354,0.149409,0.027562,0.278069,0.124186,...,0.193056,0.036329,0.060750,0.148598,0.117442,0.171481,0.009353,0.074669,0.159980,0.004751
508,0.003717,-0.343065,-0.195749,-0.139559,-0.238768,0.040010,-0.175854,-0.199708,-0.305484,-0.310527,...,-0.075011,0.020831,-0.165601,-0.186285,-0.194115,-0.233946,-0.049405,-0.102780,-0.132459,-0.328177
509,0.547733,0.551904,0.618385,0.288476,0.573452,0.405307,0.082592,0.578527,0.083675,0.472979,...,0.560347,0.890254,0.483646,0.672192,0.391036,0.324397,0.518081,0.501407,0.470276,0.551723
510,-0.107744,-0.026610,-0.038616,-0.066726,0.025246,-0.004127,0.008500,-0.128752,0.099020,-0.020051,...,-0.180705,-0.093279,-0.159255,-0.050155,0.071644,0.071283,-0.126087,-0.135820,-0.076995,-0.055523
511,-0.000145,-0.033158,0.020029,0.035586,0.074095,0.010334,0.187956,0.115864,-0.061204,0.035707,...,-0.045162,-0.068201,-0.074078,0.035937,0.033973,-0.013996,0.067880,0.067073,0.173894,-0.013018
